In [ ]:
from tensorflow import keras
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
#加载模型A并基于该模型的层创建一个新模型，重用除输出层外的所有层
model_A = keras.models.load_model("my_model_A.h5")
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

X_B, y_B = make_classification(
    n_samples=2000,  # 总样本数
    n_features=20,   # 特征数，应与模型A的输入层匹配
    n_classes=2,     # 二分类
    n_redundant=0,   # 无冗余特征
    n_informative=20, # 所有特征都有信息量
    random_state=42  # 确保每次运行结果一致
)

# 将数据分为训练集、验证集和测试集
X_temp, X_test_B, y_temp, y_test_B = train_test_split(
    X_B, y_B, test_size=0.2, random_state=42, stratify=y_B
)
X_train_B, X_valid_B, y_train_B, y_valid_B = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp # 0.25 x 0.8 = 0.2 of total
)

#由于新的输出层是随机初始化的，它会产生较大的错误（至少在前几个轮次内），因此将存在较大的错误梯度，这可能会破坏重用的权重.为了避免这种情况，我们在前几个轮次时冻结重用的层，给新层一些时间来学习合理的权重
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False
model_B_on_A.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])

#解冻重用的层，并继续进行训练来微调任务B的重用层；降低学习率可以再次避免损坏重用的权重
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4, validation_data=(X_valid_B,y_valid_B))
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True
optimizer = keras.optimizers.SGD(lr=1e-4) # the default lr is 1e-2
model_B_on_A.compile(loss="binary_crossentropy", optimizer=optimizer,
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,validation_data=(X_valid_B, y_valid_B))
model_B_on_A.evaluate(X_test_B, y_test_B)